In [1]:
using MPSGE

Build the simple TWOBYTWO model:

In [2]:
m = Model()

@parameter(m, endow, 1.0)

@sector(m, X)
@sector(m, Y)
@sector(m, U)

@commodity(m, PX)
@commodity(m, PY)
@commodity(m, PU)
@commodity(m, PL)
@commodity(m, PK)

@consumer(m, RA, benchmark = 150.)

@production(m, X, 0, 1, [Output(PX, 100)], [Input(PL, 50), Input(PK, 50)])
@production(m, Y, 0, 1, [Output(PY, 50)], [Input(PL, 20), Input(PK, 30)])
@production(m, U, 0, 1, [Output(PU, 150)], [Input(PX, 100), Input(PY, 50)])

@demand(m, RA, [Demand(PU, 150)], [Endowment(PL, :(70 * $endow)), Endowment(PK, 80.)])

MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)


Next try to solve it with an iteration limit of 0 to replicate the benchmark:

In [3]:
solve!(m, cumulative_iteration_limit=0)

Reading options file C:\Users\david\AppData\Local\Temp\jl_5A57.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (F_X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.079000


Residual. . . . . . . . 0.000000e+00


MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Solution:
  X:	1.0
  Y:	1.0
  U:	1.0
  PX:	1.0
  PY:	1.0
  PU:	1.0
  PL:	1.0
  PK:	1.0
  PX‡X:	100.0
  PY‡Y:	50.0
  PU‡U:	150.0
  PL†X:	50.0
  PK†X:	50.0
  PL†Y:	20.0
  PK†Y:	30.0
  PX†U:	100.0
  PY†U:	50.0
  RA:	150.0
  PUρRA:	150.0


Now change the value of the parameter `endow` and solve the model again:

In [4]:
set_value(endow, 1.1)
solve!(m)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             6.8995e+00             0.0e+00 (F_RA)
    1     1     0    19 5.6719e-01  1.0e+00    0.0e+00 (F_PUρRA)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 5.6719e-01           I 0.0e+00 3.5e-01 (F_PUρRA)
    1     1     3     3 3.7490e-04  1.0e+00 SO 0.0e+00 3.0e-04 (F_PK)
    2     1     4     4 1.9273e-08  1.0e+00 SO 0.0e+00 1.3e-08 (F_PL†X)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient Evaluations. . 4
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.047000
Residual. . . . . . . . 1.927322e-08


MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Solution:
  X:	1.0488088481734956
  Y:	1.038860118257079
  U:	1.0454820635822557
  PX:	1.0488566120268248
  PY:	1.0589010742434704
  PU:	1.0521941346436612
  PL:	1.0000455408083084
  PK:	1.1000500954149834
  PX‡X:	100.00000000000001
  PY‡Y:	49.99999999999999
  PU‡U:	150.0
  PL†X:	52.44044240818329
  PK†X:	47.67312946227613
  PL†Y:	21.17705705826264
  PK†Y:	28.87780507969781
  PX†U:	100.31820580255858
  PY†U:	49.68330660298152
  RA:	165.00751427543844
  PUρRA:	156.82230953733836


Now fix the `PX` variable at its benchmark value:

In [5]:
set_fixed!(PX, true)
solve!(m)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 18

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             6.8995e+00             0.0e+00 (F_RA)
    1     1     0    18 4.0468e-01  1.0e+00    0.0e+00 (F_X)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 4.0468e-01           I 0.0e+00 2.8e-01 (F_X)
    1     1     3     3 5.7858e-04  1.0e+00 SO 0.0e+00 3.4e-04 (F_X)
    2     1     4     4 1.8977e-09  1.0e+00 SO 0.0e+00 1.2e-09 (F_X)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient Evaluations. . 4
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 1.897697e-09
Postsolved residual: 1.8977e-09


MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Solution:
  X:	1.0488088481874949
  Y:	1.0388601182572046
  U:	1.0454820635823234
  PX:	1.0
  PY:	1.0095765827762655
  PU:	1.003182058025557
  PL:	0.9534625892390532
  PK:	1.0488088481827
  PX‡X:	100.0
  PY‡Y:	50.0
  PU‡U:	150.0
  PL†X:	52.44044240795569
  PK†X:	47.67312946136332
  PL†Y:	21.177057058092654
  PK†Y:	28.877805079309738
  PX†U:	100.31820580255415
  PY†U:	49.683306602985084
  RA:	157.3213272260231
  PUρRA:	156.8223095373485


Display an algebraic version of the model:

In [6]:
jm = algebraic_version(m)

Mixed complementarity problem with 19 constraints:
  Compensated Demand
    50.0 * ((PL ^ (50.0 / (50.0 + 50.0)) * PK ^ (50.0 / (50.0 + 50.0))) / PL) ^ 1.0 - PL†X      ┴  PL†X
    50.0 * ((PL ^ (50.0 / (50.0 + 50.0)) * PK ^ (50.0 / (50.0 + 50.0))) / PK) ^ 1.0 - PK†X      ┴  PK†X
    20.0 * ((PL ^ (20.0 / (20.0 + 30.0)) * PK ^ (30.0 / (20.0 + 30.0))) / PL) ^ 1.0 - PL†Y      ┴  PL†Y
    30.0 * ((PL ^ (20.0 / (20.0 + 30.0)) * PK ^ (30.0 / (20.0 + 30.0))) / PK) ^ 1.0 - PK†Y      ┴  PK†Y
    100.0 * ((PX ^ (100.0 / (100.0 + 50.0)) * PY ^ (50.0 / (100.0 + 50.0))) / PX) ^ 1.0 - PX†U  ┴  PX†U
    50.0 * ((PX ^ (100.0 / (100.0 + 50.0)) * PY ^ (50.0 / (100.0 + 50.0))) / PY) ^ 1.0 - PY†U   ┴  PY†U
  Compensated Supply
    (100.0 * PX) / sqrt(+((100.0 * PX ^ 2.0) / +100.0)) - PX‡X                                  ┴  PX‡X
    (50.0 * PY) / sqrt(+((50.0 * PY ^ 2.0) / +50.0)) - PY‡Y                                     ┴  PY‡Y
    (150.0 * PU) / sqrt(+((150.0 * PU ^ 2.0) / +150.0)) - PU‡U             